In [1]:
import numpy as np
import os
from imutils import paths
import csv
import copy
from utils import *

/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/mnt/sdd/fcastro/envs/TF/lib/python3.6/site-packages/tensorflow/python/frame

In [16]:
metadata_file = "../data/Metadatos.csv"
#data_path = "../data/COVIDGR1.0-Segmentadas"
data_path = "../data/Revisadas-Clasificadas"

hospital_dict = get_hospital_dict(data_path, metadata_file)
niveles_rale_dict, puntos_rale_dict = get_rale_dict(data_path, metadata_file)
demograf_dict = get_demograf_dict(data_path, metadata_file)

"""
for k in puntos_rale_dict.keys():
    print(str(k) + ": " + str( len(puntos_rale_dict[k]) ) )
for k in demograf_dict.keys():
    print(str(k) + ": " + str( len(demograf_dict[k]) ) )
"""

'\nfor k in puntos_rale_dict.keys():\n    print(str(k) + ": " + str( len(puntos_rale_dict[k]) ) )\nfor k in demograf_dict.keys():\n    print(str(k) + ": " + str( len(demograf_dict[k]) ) )\n'

In [6]:
def get_neg_pos(image_paths):
    neg = 0
    pos = 0
    for img in image_paths:
        label = img.split(os.sep)[-2]
        if label == 'N':
            neg = neg + 1
        else:
            pos = pos + 1
    
    return neg, pos

for k in ["San Cecilio", "MOTRIL", "ELCHE", "BAZA"]:
    image_paths = hospital_dict[k]
    neg, pos = get_neg_pos(image_paths)
    print(k + " N: " + str(neg) + " P: " + str(pos))

San Cecilio N: 449 P: 334
MOTRIL N: 29 P: 68
ELCHE N: 8 P: 24
BAZA N: 2 P: 19


In [12]:
s = 3
np.random.seed(s)
federated_data, test_data = federate_data_hospital(hospital_dict)

for i in range(len(federated_data)):
    image_paths = federated_data[i]
    neg, pos = get_neg_pos(image_paths)
    print(str(i) + " N: " + str(neg) + " P: " + str(pos))

csv_file_path = "../partitions/partition_noniid_hospital_" + str(s) + ".csv"
write_csv_file(federated_data, test_data, csv_file_path)

0 N: 346 P: 280
1 N: 23 P: 54
2 N: 8 P: 17
3 N: 2 P: 14


In [15]:
s = 3
np.random.seed(s)
federated_data, test_data = federate_data_nivelesrale(niveles_rale_dict)

for i in range(len(federated_data)):
    image_paths = federated_data[i]
    neg, pos = get_neg_pos(image_paths)
    print(str(i) + " N: " + str(neg) + " P: " + str(pos))

csv_file_path = "../partitions/partition_noniid_nivelesrale_" + str(s) + ".csv"
write_csv_file(federated_data, test_data, csv_file_path)

0 N: 63 P: 57
1 N: 118 P: 87
2 N: 153 P: 139
3 N: 67 P: 65


In [19]:
s = 3
np.random.seed(s)
federated_data, test_data = federate_data_puntosrale(puntos_rale_dict)

for i in range(len(federated_data)):
    image_paths = federated_data[i]
    neg, pos = get_neg_pos(image_paths)
    print(str(i) + " N: " + str(neg) + " P: " + str(pos))

csv_file_path = "../partitions/partition_noniid_puntosrale_" + str(s) +".csv"
write_csv_file(federated_data, test_data, csv_file_path)

0 N: 62 P: 59
1 N: 53 P: 41
2 N: 70 P: 39
3 N: 56 P: 52
4 N: 44 P: 39
5 N: 48 P: 49
6 N: 31 P: 42
7 N: 16 P: 21
8 N: 15 P: 10


In [21]:
s = 3
np.random.seed(s)
federated_data, test_data = federate_data_demograf(demograf_dict)

for i in range(len(federated_data)):
    image_paths = federated_data[i]
    neg, pos = get_neg_pos(image_paths)
    print(str(i) + " N: " + str(neg) + " P: " + str(pos))

csv_file_path = "../partitions/partition_noniid_demograf_" + str(s) + ".csv"
write_csv_file(federated_data, test_data, csv_file_path)

0 N: 87 P: 50
1 N: 133 P: 49
2 N: 34 P: 74
3 N: 68 P: 50
4 N: 45 P: 81
5 N: 24 P: 55
